In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
driver = webdriver.Chrome('/Users/kanovatio/Crawling/chromedriver')
driver.implicitly_wait(2)

/var/folders/9t/s4kfwbj54wj1sw01v7vtb68h0000gn/T/ipykernel_8621/3980472302.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/kanovatio/Crawling/chromedriver')


In [3]:
driver.get("https://movie.naver.com/")
time.sleep(1)

In [6]:
# 최초 페이지에서 영화 제목 입력해주는 코드

query_txt = input('영화제목은 무엇입니까? : ')

element = driver.find_element(By.CSS_SELECTOR,'#ipt_tx_srch')
element.send_keys(query_txt)  # 검색어 입력
element.send_keys("\n")  # 키보드 엔터 버튼

driver.find_element(By.XPATH,'//*[@id="old_content"]/ul[2]/li[1]/p').click() # 검색 버튼 클릭

영화제목은 무엇입니까? : 탑건:매버릭


In [7]:
score_list = driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[2]/div[1]')

movie_name_list=[]
movie_name = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]').text

for score in score_list : 
    
    Ascore_list = []
    Jscore_list = []
    Nscore_list = []
    
    
    
    audience_score_10 = score.find_element(By.XPATH,'//*[@id="actualPointPersentBasic"]/div/em[1]').text
    audience_score_1 = score.find_element(By.XPATH,'//*[@id="actualPointPersentBasic"]/div/em[3]').text
    audience_score_0 = score.find_element(By.XPATH,'//*[@id="actualPointPersentBasic"]/div/em[4]').text

    journalist_score_10 = score.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/div[2]/div/a/div/em[1]').text
    journalist_score_1 = score.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/div[2]/div/a/div/em[3]').text
    journalist_score_0 = score.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/div[2]/div/a/div/em[4]').text

    netizen_score_10 = score.find_element(By.XPATH,'//*[@id="pointNetizenPersentBasic"]/em[1]').text
    netizen_score_1 = score.find_element(By.XPATH,'//*[@id="pointNetizenPersentBasic"]/em[3]').text
    netizen_score_0 = score.find_element(By.XPATH,'//*[@id="pointNetizenPersentBasic"]/em[4]').text

    Ascore_list.append([audience_score_10, '.',audience_score_1, audience_score_0])
    Jscore_list.append([journalist_score_10,'.', journalist_score_1, journalist_score_0])
    Nscore_list.append([netizen_score_10,'.', netizen_score_1, netizen_score_0])

In [8]:
movie_name

'탑건: 매버릭'

In [9]:
Jscore_list

[['8', '.', '4', '4']]

In [10]:
Ascore = pd.DataFrame(data = Ascore_list, columns=['10의 자리','.','1의 자리', '소수점'])
Ascore['관람객 평점'] = Ascore['10의 자리'] + Ascore['.'] + Ascore['1의 자리'] + Ascore['소수점']
del Ascore['10의 자리']
del Ascore['1의 자리']
del Ascore['.']
del Ascore['소수점']
Ascore['영화명'] = movie_name

Jscore = pd.DataFrame(data = Jscore_list, columns=['10의 자리','.','1의 자리', '소수점'])
Jscore['기자/평론가 평점'] = Jscore['10의 자리'] + Jscore['.'] + Jscore['1의 자리'] + Jscore['소수점']
del Jscore['10의 자리']
del Jscore['1의 자리']
del Jscore['.']
del Jscore['소수점']
Jscore['영화명'] = movie_name

Nscore = pd.DataFrame(data = Nscore_list, columns=['10의 자리','.','1의 자리', '소수점'])
Nscore['네티즌 평점'] = Nscore['10의 자리'] + Nscore['.'] + Nscore['1의 자리'] + Nscore['소수점']
del Nscore['10의 자리']
del Nscore['1의 자리']
del Nscore['.']
del Nscore['소수점']
Nscore['영화명'] = movie_name

tscore = pd.merge(Ascore, Jscore, how = 'left', on ='영화명')
total_score = pd.merge(tscore, Nscore, how = 'left', on='영화명')
total_score = total_score[['영화명', '관람객 평점', '기자/평론가 평점', '네티즌 평점']]
total_score

,영화명,관람객 평점,기자/평론가 평점,네티즌 평점
0,탑건: 매버릭,9.60,8.44,9.76
